In [1]:
import nibabel as nib
import nilearn as nil
import numpy as np
import pandas as pd
import os
import os.path
from nilearn import plotting, image
from nipype.interfaces.spm import TwoSampleTTestDesign, EstimateModel, EstimateContrast, Threshold
from nipype.pipeline.engine import Workflow, Node, MapNode
import sys
sys.path.append('..')
from src.utils.data import getDataPandas, getPandas, getConfig
os.chdir('..')

In [3]:
data = getPandas('pat_data')
data_config = getConfig('data')
train_inds = data_config['indices']['train']
test_inds = data_config['indices']['test']
data_train = data.iloc[train_inds].reset_index(drop=True)
data_test = data.iloc[test_inds].reset_index(drop=True)
data_full = data.iloc[train_inds + test_inds].reset_index(drop=True)

In [4]:
train_good = data_train[data_train['CAT_MDS']==1]
train_bad = data_train[data_train['CAT_MDS']==0]
#train_good = data_full[data_full['CAT']==1]
#train_bad = data_full[data_full['CAT']==0]
img_good = train_good['FSL_SGM']
img_bad = train_bad['FSL_SGM']
img_good = img_good.apply(os.path.abspath)
img_bad = img_bad.apply(os.path.abspath)

In [5]:
age_good = train_good['AGE']
age_bad = train_bad['AGE']
sex_good = train_good['SEX']
sex_bad = train_bad['SEX']
agelist = age_good.tolist() + age_bad.tolist()
sexlist = sex_good.tolist() + sex_bad.tolist()

In [8]:
wf = Workflow(name='voxelwise', base_dir='tmp')

ttest = Node(TwoSampleTTestDesign(), name='ttest')
ttest.inputs.group1_files = img_good.tolist()
ttest.inputs.group2_files = img_bad.tolist()
ttest.inputs.covariates = [
    {'name': 'age', 'vector': agelist},
    {'name': 'sex', 'vector': sexlist},
]

model = Node(EstimateModel(), name='model')
model.inputs.estimation_method = {'Classical': 1}

contrast = Node(EstimateContrast(), name='contrast')
con1 = ('good>bad','T', ['Group_{1}','Group_{2}'],[1,-1])
con2 = ('bad>good','T', ['Group_{1}','Group_{2}'],[-1,1])
con_list = [con1, con2]
contrast.inputs.contrasts = con_list
contrast.inputs.group_contrast = True

threshold = MapNode(Threshold(
    contrast_index=1,
    use_topo_fdr=True,
    use_fwe_correction=False,
    extent_threshold=0,
    height_threshold=0.05,
    height_threshold_type='p-value',
    extent_fdr_p_threshold=0.05), name='thres', iterfield=['stat_image'])

wf.connect([
    (ttest, model, [('spm_mat_file', 'spm_mat_file')]),
    (model, contrast, [('spm_mat_file', 'spm_mat_file'), ('beta_images', 'beta_images'), ('residual_image', 'residual_image')]),
    (contrast, threshold, [('spm_mat_file', 'spm_mat_file'), ('spmT_images', 'stat_image')])
])

wf.run()

230407-10:23:56,18 nipype.workflow INFO:
	 Workflow voxelwise settings: ['check', 'execution', 'logging', 'monitoring']
230407-10:23:56,32 nipype.workflow INFO:
	 Running serially.
230407-10:23:56,33 nipype.workflow INFO:
	 [Node] Setting-up "voxelwise.ttest" in "/home/biobot/disk/ldopa/tmp/voxelwise/ttest".
230407-10:23:56,41 nipype.workflow INFO:
	 [Node] Cached "voxelwise.ttest" - collecting precomputed outputs
230407-10:23:56,41 nipype.workflow INFO:
	 [Node] "voxelwise.ttest" found cached.
230407-10:23:56,42 nipype.workflow INFO:
	 [Node] Setting-up "voxelwise.model" in "/home/biobot/disk/ldopa/tmp/voxelwise/model".
230407-10:23:56,44 nipype.workflow INFO:
	 [Node] Cached "voxelwise.model" - collecting precomputed outputs
230407-10:23:56,44 nipype.workflow INFO:
	 [Node] "voxelwise.model" found cached.
230407-10:23:56,44 nipype.workflow INFO:
	 [Node] Setting-up "voxelwise.contrast" in "/home/biobot/disk/ldopa/tmp/voxelwise/contrast".
230407-10:23:56,47 nipype.workflow INFO:
	 [No

stty: 标准输入: 对设备不适当的 ioctl 操作


230407-10:24:11,49 nipype.workflow INFO:
	 [Node] Finished "_thres1", elapsed time 7.456399s.


stty: 标准输入: 对设备不适当的 ioctl 操作


In [4]:
data = getPandas('hc_data')
label_list = list(data['IMG_ROOT'] + os.sep + 'cat12' + os.sep + 'label' + os.sep + 'catROI_raw.xml')